In [73]:
import importlib
import copy
import pickle
# --------------------
from sklearn.utils import gen_batches, check_array
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.utils.validation import FLOAT_DTYPES
import numpy as np
from numpy.linalg import norm,inv,matrix_rank, pinv
# --------------------
from skopt.space import Integer, Real
from skopt.utils import use_named_args
from skopt import gp_minimize
# --------------------
import h5py
import Code.OLPLS
import warnings
warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [68]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)

    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )

In [ ]:
with h5py.File('./data/merra2_t.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)

    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]

n_fold = 74
test_size=30    

In [ ]:
with h5py.File('./data/TW_PM25.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)
        
    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
n_fold = 17
test_size=30

In [64]:
def dump_PLS(PLS, fn):
    np.savez(fn,
             n=PLS.n,
             n_comp=PLS.n_components,
             W = PLS.W,
             y_loadings = PLS.C,
             x_loadings = PLS.P
             )

def predict(dat, X, n_comp):
    X = check_array(X, copy=copy, dtype=FLOAT_DTYPES)
    W = dat['W'][:,:n_comp]
    x_loadings = dat['x_loadings'][:,:n_comp]
    y_loadings = dat['y_loadings'][:,:n_comp]
    x_rotaions = np.dot(W,
                        pinv(np.dot(x_loadings.T, W))
    )

    coef = np.dot(x_rotaions, y_loadings.T)
    ypred = np.dot(X, coef)
    return ypred


In [70]:
print(X_train.shape)
print(Y_train.shape)

tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break

print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break

(877, 1800)
(877, 1800)
Fold 0:
  Train: len=367
  Test:  len=30
Fold 1:
  Train: len=397
  Test:  len=30
Fold 2:
  Train: len=427
  Test:  len=30
Fold 3:
  Train: len=457
  Test:  len=30
Fold 4:
  Train: len=487
  Test:  len=30
  Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187

In [71]:
n_comp_max=16
n_comp_list=list(range(1,n_comp_max+1))
mu_list= [1e-5, 1e-7, 1e-9]
amnesic_list= [1e-2] + list(np.arange(0.1,1.0,0.4)) + [0.99]
# n_comp_max=100
# mu_list= [1e-5, 1e-6, 1e-7, 1e-8]
# amnesic_list= [1e-2] + list(np.arange(0.1,1.0,0.1)) + [0.99]

print("n_comp:", n_comp_list, "; len=", len(n_comp_list))
print("mu_list:", mu_list, "; len=", len(mu_list))
print("amnesic_list:", "; len=", len(amnesic_list))



n_comp: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] ; len= 16
mu_list: [1e-05, 1e-07, 1e-09] ; len= 3
amnesic_list: ; len= 5


In [37]:
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS

scores = []
params = []

tim_tot.tic()
for mu in mu_list:
    for amnesic in amnesic_list:
        PLS = OLPLS(n_components=n_comp_max, mu=mu, amnesic=amnesic)
        tim.tic()
        score_loc = np.zeros((n_fold, len(n_comp_list)))
        for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
            PLS.fit(X_train[train_index], Y_train[train_index])
            y_true = Y_train[test_index]
            try:
                for j,n_comp in enumerate(n_comp_list):
                    y_pred = PLS.predict(X_train[test_index], n_comp)
                    score_loc[i,j] = RMSE(y_true, y_pred)
            except Exception as e:
                print("-"*40)
                print(f"mu={mu:.1e}; amnesic={amnesic:.2f}; "
                      +f"fold: {i+1:02d}")
                print(e)
                print("-"*40, "\n")
                score_loc[:,:] = np.nan
                break

        wrk = score_loc.mean(axis=0)
        for j,n_comp in enumerate(n_comp_list):
            scores.append(wrk[j])
            params.append({'mu': mu, 'amnesic': amnesic, 'n_comp': n_comp})

        if (i%5==0):
            print(f"mu={mu:.1e}; amnesic={amnesic:.2f}; "
                  +f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
            tim.tic()


print(scores)
best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")


----------------------------------------
mu=1.000000e-05; amnesic=0.01; fold: 01
illegal value in 4th argument of internal gesdd
----------------------------------------
mu=1.000000e-05; amnesic=0.01; fold: 01, elapsed time: 39.0s


----------------------------------------
mu=1.000000e-05; amnesic=0.10; fold: 01
illegal value in 4th argument of internal gesdd
----------------------------------------
mu=1.000000e-05; amnesic=0.10; fold: 01, elapsed time: 38.8s


----------------------------------------
mu=1.000000e-05; amnesic=0.50; fold: 01
illegal value in 4th argument of internal gesdd
----------------------------------------
mu=1.000000e-05; amnesic=0.50; fold: 01, elapsed time: 38.8s


----------------------------------------
mu=1.000000e-05; amnesic=0.90; fold: 01
illegal value in 4th argument of internal gesdd
----------------------------------------
mu=1.000000e-05; amnesic=0.90; fold: 01, elapsed time: 38.6s


----------------------------------------
mu=1.000000e-05; amnesic=0.99; fold: 01
illegal value in 4th argument of internal gesdd
----------------------------------------
mu=1.000000e-05; amnesic=0.99; fold: 01, elapsed time: 38.6s


...

----------------------------------------
mu=1.000000e-07; amnesic=0.90; fold: 01
illegal value in 4th argument of internal gesdd
----------------------------------------
mu=1.000000e-07; amnesic=0.90; fold: 01, elapsed time: 38.7s


----------------------------------------
mu=1.000000e-07; amnesic=0.99; fold: 01
illegal value in 4th argument of internal gesdd
----------------------------------------
mu=1.000000e-07; amnesic=0.99; fold: 01, elapsed time: 38.6s


----------------------------------------
mu=1.000000e-09; amnesic=0.01; fold: 11
illegal value in 4th argument of internal gesdd
----------------------------------------
mu=1.000000e-09; amnesic=0.01; fold: 11, elapsed time: 73.6s


----------------------------------------
mu=1.000000e-09; amnesic=0.10; fold: 11
illegal value in 4th argument of internal gesdd
----------------------------------------
mu=1.000000e-09; amnesic=0.10; fold: 11, elapsed time: 75.4s


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 8.468475622098396, 8.879133521134285, 9

In [55]:
for i in range(-2,3):
    ind = best_ind+9+16*i
    print(params[ind], scores[ind])

{'mu': 1e-09, 'amnesic': 0.01, 'n_comp': 10} nan
{'mu': 1e-09, 'amnesic': 0.1, 'n_comp': 10} nan
{'mu': 1e-09, 'amnesic': 0.5, 'n_comp': 10} 9.463073043678241
{'mu': 1e-09, 'amnesic': 0.9, 'n_comp': 10} 9.461052464766222
{'mu': 1e-09, 'amnesic': 0.99, 'n_comp': 10} 9.45751651718039


In [121]:
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS

space  = [Real(1e-9, 1e-5, name='mu', prior="log-uniform"),
          Real(0, 1, name='amnesic')]

@use_named_args(space)
def Comp_Model_Score(mu, amnesic):
    PLS=OLPLS(n_components=n_comp_max, mu=mu, amnesic=amnesic)
    print(f"__mu={PLS.mu:.4e}, amnesic={PLS.amnesic:.2f}")
    scores_loc = np.zeros((n_fold,n_comp_max))
    tim.tic()
    for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
        PLS.fit(X_train[train_index], Y_train[train_index])
        if(i%10==0):
            print(f"____fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
            tim.tic()

        y_true = Y_train[test_index]
        for j,n_comp in enumerate(range(1,n_comp_max+1)):
            try:
                y_pred = PLS.predict(X_train[test_index], n_comp)
                scores_loc[i,j] = RMSE(y_true, y_pred)
            except Exception as e:
                scores_loc[i,j] = np.nan

    scores = scores_loc.mean(axis=0)
    scores_min = np.nanmin(scores)

    if(np.isnan(scores_min)):
        scores_min=9.9999e+50
        print("__failure")
    else:
        print("scores: ", scores[:5])
        print(f"__best n_comp: {np.nanargmin(scores)+1:d}")

    return scores_min


In [122]:
tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=20,
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best amnesic = {res_gp.x[0]}")
print(f"Best mu = {res_gp.x[1]}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")


Iteration No: 1 started. Evaluating function at random point.
__mu=2.3517e-07, amnesic=0.84


____fold: 01, elapsed time: 38.7s


____fold: 11, elapsed time: 32.1s


__failure
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 90.0656
Function value obtained: 999990000000000010022458331596220993686065066803200.0000
Current minimum: 999990000000000010022458331596220993686065066803200.0000
Iteration No: 2 started. Evaluating function at random point.
__mu=2.7026e-06, amnesic=0.85


____fold: 01, elapsed time: 38.4s


...

Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 94.8529
Function value obtained: 8.0910
Current minimum: 7.9019
Iteration No: 20 started. Searching for the next optimal point.
__mu=1.5276e-08, amnesic=0.06


____fold: 01, elapsed time: 38.6s


____fold: 11, elapsed time: 32.6s


scores:  [8.82438832        nan        nan        nan        nan]
__best n_comp: 1


Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 91.1129
Function value obtained: 8.8244
Current minimum: 7.9019
----------------------------------------
Best amnesic = 3.6425298685704783e-09
Best mu = 0.8700872583584366
Best score = 7.9019
total time = 1839.3s


In [102]:
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS

PLS=OLPLS(n_components=n_comp_max, mu=res_gp.x[1], amnesic=res_gp.x[0])
# PLS=OLPLS(n_components=n_comp_max, mu=1e-9, amnesic=0.5)
print(f"__mu={PLS.mu:.2f}, amnesic={PLS.amnesic:.2e}")
scores_loc = np.zeros((n_fold,n_comp_max))
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    y_true = Y_train[test_index]
    for j,n_comp in enumerate(range(1,n_comp_max+1)):
        try:
            y_pred = PLS.predict(X_train[test_index], n_comp)
            scores_loc[i,j] = RMSE(y_true, y_pred)
        except Exception as e:
            scores_loc[i,j] = np.nan

    if(i%10==0):
        print(f"____fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()

scores = scores_loc.mean(axis=0)
scores_min = np.nanmin(scores)
print("scores: ", scores[:])
print(f"__best n_comp: {np.nanargmin(scores)+1:d}")
if(np.isnan(scores_min)):
    scores_min=9.9999e+100

__mu=0.50, amnesic=1.00e-09


____fold: 01, elapsed time: 38.9s


____fold: 11, elapsed time: 32.0s


scores:  [8.56280844 8.91785021 9.10520643 9.21845875 9.2934175  9.34618831
 9.3851262  9.41494808 9.43848354 9.45751652 9.47322052 9.48639632
 9.49760793 9.50726362 9.51566601 9.52304412]
__best n_comp: 1


In [104]:
print("scores: ", scores[9])    

scores:  9.45751651718039


In [ ]:
n_calls = int(np.log(n_comp_max)) + 1
print(f"n_calls = {n_calls:d}")
n_calls *= 3

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=max(n_calls,10), 
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best param = {res_gp.x[0]:02d}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")


In [ ]:
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS

scores = []
params = []
n_comp_list=list(range(100,401,50))

tim_tot.tic()
for n_comp in n_comp_list:
    tim.tic()
    pls=OLPLS(n_components=n_comp)
    scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
    params.append({'n_components': n_comp})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")
